In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# import stockAI_v3 as sai

# 1. Data Collection
- 함수 변경해야 함
- 크롤링 후 데이터 저장 방법: github 업데이트
- 국내 KRX
- 해외 investing.com


- fdr 데이터 수집 2001~ 

## 1.1 Load tickers 

In [3]:
lst_tickers = get_ticker(markets=['KSC', 'KOE']) # KSC: KOSPI, KOE: KOSDAQ
print(len(lst_tickers))
print(lst_tickers[:5])

NameError: name 'get_ticker' is not defined

## 1.2 Load stock data

# 2. Data Preprocessing

## 2.1 Add indicator

In [ ]:
stock_data = pd.read_csv('raw_data_ex_1004.csv')

In [ ]:
stock_data

In [ ]:
check_index = ['MA5', 'MA20', 'MA60','MA120', 
             'trading_value','next_change',
             'MFI','ADI','OBV','CMF','FI','EOM_EMV','VPT','NVI','VMAP',
             'BHB','BLB','KCH','KCL','KCM','DCH','DCL','DCM','UI',
             'SMA','EMA','WMA','MACD','VIneg','VIpos','TRIX','MI','CCI','DPO','KST','Ichimoku','ParabolicSAR','STC',
             'RSI','SRSI','TSI','UO','SR','WR','AO','KAMA','ROC','PPO','PVO']

check_df = sai.add_index(data=stock_data, index_list=check_index)
check_df

## 2.2 Data Scaling

In [ ]:
import pandas as pd
check_df = pd.read_csv('../_common/processed_data_ex_1004.csv')

In [ ]:
check_scaled_KR = sai.scaling(data=check_df, scaler_type="div-close", window_size=None)
check_scaled_KR

## 2.2 Data transformation in time series
Set day=10 to convert to time series data with 10 days per row.

In [ ]:
df_time_series = sai.time_series(check_scaled_KR, day=10)

In [ ]:
df_time_series

# 3. Model training and evaluation

## 3.1 Data seperation to train, test dataset

In [ ]:
data = check_df # Data before scaling
data_scaled = check_scaled_KR # Data after scaling

# lst_date = [[['2017-01-01', '2019-12-31'], ['2020-01-01', '2020-12-31']], 
#            [['2018-01-01', '2020-12-31'], ['2021-01-01', '2021-12-31']]] # 2-fold cv

# Data seperation to train, test dataset
train_data = data[(data['Date'] >= '2017-01-01') & (data['Date'] <= '2020-12-31')]
test_data = data[(data['Date'] >= '2021-01-01') & (data['Date'] <= '2021-12-31')]

# Scaled data seperation into train, test dataset
train_data_scaled = data_scaled[(data_scaled['Date'] >= '2017-01-01') & (data_scaled['Date'] <= '2020-12-31')]
test_data_scaled = data_scaled[(data_scaled['Date'] >= '2021-01-01') & (data_scaled['Date'] <= '2021-12-31')]

print(train_data.shape, test_data.shape)
print(train_data_scaled.shape, test_data_scaled.shape)

## 3.2 Trader definition
Add a model, time series model

In [ ]:
# list to store trader objects
lst_trader = [] 

#### 1) SuwonLab XGBoost

In [ ]:
from xgboost import XGBClassifier

# condition acquisition object
b1_sw = sai.conditional_buyer() 

def sampling1(df): # generating condition function
    condition1 = (-0.3 <= df.Change) & (df.Change <= 0.3) # Eliminate exceptions exceeding the upper and lower limits
    condition2 = df.trading_value >= 1000000000 # Volatility condition 1: Transaction amount of more than 1 billion won
    condition3 = (-0.05 >= df.Change) | (0.05 <= df.Change) # Volatility condition 2: Today's stock price change rate is more than 5%
    condition = condition1 & condition2 & condition3
    return condition

b1_sw.condition = sampling1 # Save Condition Function


# Create Machine Learning Purchase Object
b2_sw = sai.machinelearning_buyer()  

def transform(data):
    # Change to two-dimensional structure
    pass

b2_sw.data_transform = transform


scale_pos_weight = round(72/28 , 2)
b2_sw.algorithm = XGBClassifier(random_state = 42,
                   n_jobs=30,
                   scale_pos_weight=scale_pos_weight,
                   learning_rate=0.1,
                   max_depth=4,
                   n_estimators=1000,
                   )  
b2_sw.threshold = 0.8


# Create a sales object
sell_all = sai.SubSeller() ## (Add)


# Trader object 
t1 = sai.Trader()
t1.name = 'SuwonLabXGboost'
t1.label = 'class&0.02' ## (Add)
t1.buyer = sai.Buyer([b1_sw, b2_sw]) # [ conditional buyer, machinelearning buyer ] 
t1.seller = sai.Seller(sell_all)
t1.scaling = 'div-close'

lst_trader.append(t1)

#### 2) GyeonggiLab LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

# condition acquisition object
b1_gg = sai.conditional_buyer()

def sampling2(df): # generating condition function
    condition1 = (-0.3 <= df.Change) & (df.Change <= 0.3) # Eliminate exceptions exceeding the upper and lower limits
    condition2 = df.trading_value >= 1000000000 # Volatility condition 1: Transaction amount of more than 1 billion won
    condition3 = (-0.05 >= df.Change) | (0.05 <= df.Change) # Volatility condition 2: Today's stock price change rate is more than 5% 
    condition = condition1 & condition2 & condition3
    return condition

b1_gg.condition = sampling2


# Create Machine Learning Purchase Object
b2_gg = sai.machinelearning_buyer()  
b2_gg.algorithm = LogisticRegression()
b2_gg.threshold = 0.5


# Create a sales object
sell_all = sai.SubSeller() ## (Add)


# Trader object    
t2 = sai.Trader()
t2.name = 'GyeonggiLabLogisticRegression'
t2.label = 'class&0.02' ## (Add)
t2.buyer = sai.Buyer([b1_gg, b2_gg]) # [ conditional buyer, machinelearning buyer ] 
t2.seller = sai.Seller(sell_all)
t2.scaling = 'div-close'

lst_trader.append(t2)

#### 3) Pororo LightGBM

In [ ]:
from lightgbm import LGBMClassifier

# condition acquisition object 
b1_pr = sai.conditional_buyer()

def sampling3(df): # generating condition function
    condition1 = (-0.3 <= df.Change) & (df.Change <= 0.3) # Eliminate exceptions exceeding the upper and lower limits
    condition2 = df.trading_value >= 1000000000 # Volatility condition 1: Transaction amount of more than 1 billion won
    condition3 = (-0.05 >= df.Change) | (0.05 <= df.Change) # Volatility condition 2: Today's stock price change rate is more than 5%
    condition = condition1 & condition2 & condition3
    return condition

b1_pr.condition = sampling3


# Create Machine Learning Purchase Object
b2_pr = sai.machinelearning_buyer()

scale_pos_weight = round(72/28 , 2)
params = {  'random_state' : 42,
            'scale_pos_weight' : scale_pos_weight,
            'learning_rate' : 0.1, 
            'num_iterations' : 1000,
            'max_depth' : 4,
            'n_jobs' : 30,
            'boost_from_average' : False,
            'objective' : 'binary' }

b2_pr.algorithm =  LGBMClassifier( **params )
b2_pr.threshold = 0.8


# Create a sales object
sell_all = sai.SubSeller() ## (Add)


# Trader object    
t3 = sai.Trader()
t3.name = 'PororoLightGBM'
t3.label = 'class&0.02' ## (Add)
t3.buyer = sai.Buyer([b1_pr, b2_pr]) # [ conditional buyer, machinelearning buyer ] 
t3.seller = sai.Seller(sell_all)
t3.scaling = 'div-close'

lst_trader.append(t3)

## 3.3 Create and store required datasets 
**[Types of Data Stored]**  
- Original dataset (training/test, independent/dependent variables_regression/dependent variables_classification)
- Standardized dataset (teaching/testing, independent/dependent variables_regression/dependent variables_classification)

In [ ]:
sai.save_dataset(lst_trader, train_data, test_data, train_data_scaled, test_data_scaled)

## 3.4 Model Training and Testing

#### 1) Model Training

In [ ]:
sai.trader_train(lst_trader) 

#### 2) Model evaluation and threshold setting

In [ ]:
sai.get_eval_by_threshold(lst_trader)

#### 3) Profitability verification histogram

In [ ]:
sai.histogram_plot(lst_trader)

## 3.5 Calculate the rate of return

#### 1) Preparation of Transaction log

In [ ]:
df_signal_all = sai.decision(lst_trader, dtype='test')

In [ ]:
df_signal_all

#### 2) simulation (calculation the rate of return)

In [ ]:
df_history_all = sai.simulation(df_signal_all, init_budget=10000000, init_stock={})

In [ ]:
df_history_all

#### 3) Leaderboard

In [ ]:
sai.leaderboard(df_history_all)

#### 4) Visualization of returns

In [ ]:
sai.yield_plot(df_history_all)